# Compression Ratio Side-Channel Attacks
Requirement for attack:  
1. Partial plaintext knowledge and
2. Partial plaintext control and
3. Access to a compression oracle

A compression oracle is one that takes input plaintext and returns the length of the compressed output. An example of the output length is the output length of HTTPS requests

## Stream Cipher Case

In [1]:
import zlib
from Crypto.Cipher import Salsa20
def oracle(plaintext: str) -> int:
    """
        Oracle that returns the length of the encryption of the compression of the plaintext
        oracle(P) -> length(encrypt(compress(format_request(P))))
    """
    formatted_request = f"""POST / HTTP/1.1
Host: hapless.com
Cookie: sessionid=TmV2ZXIgcmV2ZWFsIHRoZSBXdS1UYW5nIFNlY3JldCE=
Content-Length: {len(plaintext)}
{plaintext}"""
    compressed_request = zlib.compress(formatted_request.encode())
    cipher = Salsa20.new(key=b"YELLOW SUBMARINE", nonce=b"\x00" * 8)
    return len(cipher.encrypt(compressed_request)) 

In [2]:
## Attack
from utils import xor, get_blocks
from string import ascii_letters, digits

DICTIONARY = ascii_letters + digits + "="
answer = 'Cookie: sessionid=TmV2ZXIgcmV2ZWFsIHRoZSBXdS1UYW5nIFNlY3JldCE='

def retrieve_secret(oracle: callable, known_secret: str) -> str:
    base_length = oracle(known_secret)
    secret = known_secret
    for i in range(1, 100):
        for c in DICTIONARY:
            guess = secret + c
            length = oracle(guess)
            if length == base_length:
                secret = guess
                base_length = length
    return secret

base_known = "Cookie: sessionid="
secret = retrieve_secret(oracle, base_known)
print(f'{secret}\nValid:\t{secret == answer}')

Cookie: sessionid=TmV2ZXIgcmV2ZWFsIHRoZSBXdS1UYW5nIFNlY3JldCE=
Valid:	True


## AES CBC Case:

The trick here is to known how many pads to append to the plaintext to cause the size to increase

In [3]:
import zlib
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad
def oracle(plaintext: str) -> int:
    """
        Oracle that returns the length of the encryption of the compression of the plaintext
        oracle(P) -> length(encrypt(compress(format_request(P))))
    """
    formatted_request = f"""POST / HTTP/1.1
Host: hapless.com
Cookie: sessionid=TmV2ZXIgcmV2ZWFsIHRoZSBXdS1UYW5nIFNlY3JldCE=
Content-Length: {len(plaintext)}
{plaintext}"""
    compressed_request = zlib.compress(formatted_request.encode())
    cipher = AES.new(key=b"YELLOW SUBMARINE", mode=AES.MODE_CBC, iv=b"\x00" * 16)
    return len(cipher.encrypt(pad(compressed_request, 16)))

In [4]:
## Attack
from utils import xor, get_blocks
from string import ascii_letters, digits
from padding import pkcs7_pad

DICTIONARY = ascii_letters + digits + "="
answer = 'Cookie: sessionid=TmV2ZXIgcmV2ZWFsIHRoZSBXdS1UYW5nIFNlY3JldCE='
def retrieve_secret(oracle: callable, known_secret: str) -> str:
    base_length = oracle(known_secret)
    secret = known_secret
    for i in range(1, 100):
        for c in DICTIONARY:
            guess = secret + c
            length = oracle(guess + DICTIONARY[0:6])
            #print(guess + DICTIONARY[0:6])
            #print(f'guess: {guess}, length: {length}')
            if length == base_length:
                secret = guess
                base_length = length
    return secret

base_known = "Cookie: sessionid="
secret = retrieve_secret(oracle, base_known)
print(f'{secret}\nValid:\t{secret == answer}')

Cookie: sessionid=TmV2ZXIgcmV2ZWFsIHRoZSBXdS1UYW5nIFNlY3JldCE=
Valid:	True
